In [1]:
#Add repo path to the system path
from pathlib import Path
import os, sys
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
    print(repo_path)
    
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None
exp_path = Path.cwd().resolve() # path to the experiment folder
print(f"Repo Path: {repo_path}")
print(f"Experiment Path: {exp_path}")

/home/benet/tfg
Repo Path: /home/benet/tfg
Experiment Path: /home/benet/tfg/dataset


In [2]:
#Libraries
import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.transforms import (
    Compose,
    Resize,
    CenterCrop,
    ToTensor,
    Normalize,
    InterpolationMode,
)
from diffusers import AutoencoderKL
from PIL import Image

# import the MRIDataset class from the dataset folder
from dataset.build_dataset import MRIDataset

np.random.seed(17844)
torch.manual_seed(17844)


In [ ]:
resolution = 256

# Dataset loading
data_dir = repo_path / "/home/benet/data/VH2D/images/flair"
latent_dir = repo_path / "/home/benet/data/VH2D/images/latent_flair"
# create latent directory if it does not exist
latent_dir.mkdir(exist_ok=True)

# Define the transformations to apply to the images
preprocess = Compose(
    [
        Resize(resolution, interpolation= InterpolationMode.BILINEAR), #getattr(InterpolationMode, config['processing']['interpolation'])),  # Smaller edge is resized to 256 preserving aspect ratio
        CenterCrop(resolution),  # Center crop to the desired squared resolution
        #RandomHorizontalFlip(),  # Horizontal flip may not be a good idea if we want generation only one laterality
        ToTensor(),  # Convert to PyTorch tensor
        Normalize(mean=[0.5], std=[0.5]),  # Map to (-1, 1) as a way to make data more similar to a Gaussian distribution
    ]
)


image_files = [f for f in os.listdir(data_dir) if f.endswith(".png")]
print(f"Found {len(image_files)} images in the dataset folder")

Found 570 images in the dataset folder


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

pipeline_dir = repo_path / 'results/pipelines' / 'fintuned_vae'
print(f"Pipeline directory: {pipeline_dir}")

Device: cuda
Pipeline directory: /home/benet/tfg/results/pipelines/fintuned_vae


In [ ]:
resolution = 256

# Dataset loading
data_dir = repo_path / "/home/benet/data/VH2D/images/flair"
latent_dir = repo_path / "/home/benet/data/VH2D/images/latent_flair"
# create latent directory if it does not exist
latent_dir.mkdir(exist_ok=True)

# Define the transformations to apply to the images
preprocess = Compose(
    [
        Resize(resolution, interpolation= InterpolationMode.BILINEAR), #getattr(InterpolationMode, config['processing']['interpolation'])),  # Smaller edge is resized to 256 preserving aspect ratio
        CenterCrop(resolution),  # Center crop to the desired squared resolution
        #RandomHorizontalFlip(),  # Horizontal flip may not be a good idea if we want generation only one laterality
        ToTensor(),  # Convert to PyTorch tensor
        Normalize(mean=[0.5], std=[0.5]),  # Map to (-1, 1) as a way to make data more similar to a Gaussian distribution
    ]
)


image_files = [f for f in os.listdir(data_dir) if f.endswith(".png")]
print(f"Found {len(image_files)} images in the dataset folder")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

pipeline_dir = repo_path / 'results/pipelines' / 'fintuned_vae'
print(f"Pipeline directory: {pipeline_dir}")


vae = AutoencoderKL.from_pretrained(pipeline_dir)
vae.to(device).eval()

with torch.no_grad():
    for image_file in image_files:
        img_path = os.path.join(data_dir, image_file)
        #remove .png extension and add latent in the image name
        image_file = image_file.split('.')[0] + "_latent"
        image = Image.open(img_path).convert("RGB")

        image = preprocess(image).unsqueeze(0).to(device)
        latent = vae.encode(image).latent_dist.sample().squeeze(0)
        print(latent.shape)
        # ax.imshow(latent[j].cpu().numpy(), cmap="gray")
        # save the four latent images
        latent = latent.cpu().numpy()
        for j in range(4):
            plt.imsave(latent_dir / f"{image_file}_{j}.png", latent[j], cmap="gray")        

torch.Size([4, 32, 32])
